In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Read the CSV file
data = pd.read_csv("/content/drive/MyDrive/FaceRecognition/fer2013.csv")

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
# Extract pixel values and emotions
pixels = data['pixels'].apply(lambda x: np.array(x.split(), dtype=int))
emotions = data['emotion']
usage = data['Usage']  # Assuming there's a column named 'Usage'

In [ ]:
# Reshape the pixel arrays to 2D arrays
pixels_2d = pixels.apply(lambda x: np.reshape(x, (48, 48)))

In [ ]:
# Stack the 2D arrays along a new axis to create a 3D array
pixels_2d = np.stack(pixels_2d)

In [ ]:
pixels_2d.shape

(35887, 48, 48)

In [ ]:
import cv2

original_shape = (35887, 48, 48)

new_shape = (35887, 224, 224, 3)

# Create an empty array to store the resized and converted images
resized_images = np.zeros(new_shape, dtype=np.uint8)

# Resize each image and convert to 3-channel
for i in range(original_shape[0]):
    # Resize the image to (224, 224)
    resized_image = cv2.resize(pixels_2d[i].astype(np.uint8), (224, 224))

    # Convert grayscale image to 3-channel image
    resized_image = cv2.cvtColor(resized_image, cv2.COLOR_GRAY2RGB)

    # Store the resized and converted image
    resized_images[i] = resized_image

# Check the shape of the resized images
print("Resized images shape:", resized_images.shape)


Resized images shape: (35887, 224, 224, 3)


In [ ]:
# Normalize the resized images using a for loop
for i in range(resized_images.shape[0]):
    resized_images[i] = resized_images[i].astype(np.float32) / 255.0

# Check the shape of the resized images
print("Resized images shape:", resized_images.shape)

Resized images shape: (35887, 224, 224, 3)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(resized_images, emotions, test_size=0.2, random_state=42)

# **Transfer Learning Model**

In [ ]:
model = tf.keras.applications.MobileNetV2()

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [ ]:
base_input = model.layers[0].input

In [ ]:
base_output = model.layers[-2].output

In [ ]:
base_output

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_output)

In [ ]:
final_output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_2')>

In [ ]:
new_model = keras.Model(inputs = base_input, outputs = final_output)

In [ ]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

In [ ]:
new_model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
new_model.fit(resized_images,emotions, epochs = 50)

Epoch 1/50
449/449 [==============================] - 150s 63ms/step - loss: 1.7744 - accuracy: 0.3780 - val_loss: 1.7035 - val_accuracy: 0.3083
Epoch 2/50
449/449 [==============================] - 157s 60ms/step - loss: 1.4755 - accuracy: 0.4295 - val_loss: 1.7827 - val_accuracy: 0.3632
Epoch 3/50
449/449 [==============================] - 157s 60ms/step - loss: 1.3200 - accuracy: 0.5944 - val_loss: 1.3818 - val_accuracy: 0.4766
Epoch 4/50
449/449 [==============================] - 157s 61ms/step - loss: 1.2308 - accuracy: 0.6352 - val_loss: 1.4232 - val_accuracy: 0.4752
Epoch 5/50
449/449 [==============================] - 156s 58ms/step - loss: 1.1666 - accuracy: 0.5556 - val_loss: 1.2586 - val_accuracy: 0.5241
Epoch 6/50
449/449 [==============================] - 156s 58ms/step - loss: 1.1109 - accuracy: 0.5806 - val_loss: 1.4154 - val_accuracy: 0.4802
Epoch 7/50
449/449 [==============================] - 156s 58ms/step - loss: 1.0572 - accuracy: 0.5998 - val_loss: 1.1642 - val_ac

In [ ]:
score = new_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.3921351432800293
Test accuracy: 0.8184954237937927
